<a href="https://colab.research.google.com/github/sarahfernandadc/Agente-de-treinos-personalizados/blob/main/AgenteDeTreinosPersonalizados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar Framework ADK de agentes do Google
!pip install -q google-adk

In [ ]:
# Configura a API Key do Google Gemini
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = " userdata.get('GOOGLE_API_KEY')"

In [ ]:
# Importações das bibliotecas e módulos necessários
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap  # Para formatar melhor a saída de texto
from IPython.display import display, Markdown  # Para exibir texto formatado no Colab
import requests  # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

from datetime import datetime
from typing import Dict, List
from textwrap import dedent
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória e uma nova sessão
    session_service = InMemorySessionService()
    session = session_service.create_session(
        app_name=agent.name, user_id="user1", session_id="session1"
    )
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera pelos eventos retornados durante a execução do agente para obter a resposta final
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# ========================================
# 🏋️ Agente 1 – Gerador de Plano de Treino
# ========================================

def agente_treino(perfil_usuario):
    plano = Agent(
        name="agente_gerador_treino",
        model="gemini-2.0-flash",
        instruction=dedent("""
            Você é um personal trainer especializado.
            Com base no perfil abaixo, crie um plano de treino semanal.
            Use formatação clara e acessível, com grupos musculares, exercícios, séries, repetições e sugestões de descanso.
            Evite copiar planos genéricos. Personalize de acordo com os dados fornecidos.
            Quero que o treino seja enviado em formato de tabela para facilitar a visualização.
            Neste momento desejo uma resposta limpa e sem informações sobre como fazer o exercicio ou intruções extras, a resposta deve ser apenas uma tabela com
            Dia, Grupo Muscular, Exercício,	Séries,	Repetições,	Descanso
        """),
        description="Agente que cria plano de treino com base no perfil do usuário."
    )

    entrada = dedent(f"""
    Nome: {perfil_usuario['nome']}
    Objetivo: {perfil_usuario['objetivo']}
    Nível: {perfil_usuario['nivel']}
    Frequência semanal: {perfil_usuario['frequencia']}
    Equipamento disponível: {perfil_usuario['equipamento']}
    Sexo: {perfil_usuario['sexo']}
    Altura: {perfil_usuario['altura']} m
    Peso: {perfil_usuario['peso']} kg
    """)

    resposta = call_agent(plano, entrada)
    return resposta

In [ ]:
# ===============================================
# 🔍 Agente 2 – Buscador de Execução de Exercício
# ===============================================

def agente_execucao_exercicio(nome_exercicio):
    executor = Agent(
        name="agente_execucao_exercicio",
        model="gemini-2.0-flash",
        instruction=dedent("""
            Você é um especialista em musculação e biomecânica.
            Ao receber o nome de um exercício, forneça uma explicação clara e objetiva de como executá-lo corretamente.
            Indique postura, posicionamento dos pés/mãos, possíveis variações e cuidados.
            Seja direto e prático, com foco em orientação para leigos.
            Além disso, ao final da explicação forneça uma sugestão de vídeo do YouTube sobre o exercício.
        """),
        description="Agente que explica como executar corretamente exercícios de musculação."
    )

    entrada = f"Como executar corretamente o exercício: {nome_exercicio}?"

    resposta = call_agent(executor, entrada)
    return resposta


In [ ]:
# ========================================
# 🔁 Agente 3 – Substituição de Exercício
# ========================================

def agente_substituicao_exercicio(exercicio, nivel, equipamento):
    substituidor = Agent(
        name="agente_substituidor_exercicio",
        model="gemini-2.0-flash",
        instruction=dedent(f"""
            Você é um treinador especializado em variações e substituições de exercícios.

            Para o exercício fornecido, forneça:
            - De 2 a 3 substituições ou variações viáveis
            - Indique para qual local são indicadas (casa ou academia)
            - Informe o grupo muscular trabalhado (caso o usuário não informe)
            - Se possível, dê opções com e sem equipamentos

            Adapte as sugestões ao nível do usuário ({nivel}) e aos equipamentos disponíveis ({equipamento}).

            Exemplo de resposta:
            📌 Exercício: Supino reto com barra
            🏋️ Substituições:
            - Flexão de braço elevada (para casa, iniciante)
            - Supino com halteres no chão (casa ou academia)
            - Máquina peck-deck (para peitoral, academia)
            💡 Grupo muscular: Peitoral
        """)
    )

    entrada = f"Exercício: {exercicio}; Nível: {nivel}; Equipamento: {equipamento}"

    resposta = call_agent(substituidor, entrada)
    return resposta


In [ ]:
def menu_principal():

    while True:
        print("\n🤖 Bem-vindo ao TreinIA – Agente Inteligente de Treinos 💪")
        print("Escolha uma opção:")
        print("1️⃣  Gerar plano de treino personalizado")
        print("2️⃣  Saber como executar um exercício")
        print("3️⃣  Substituir um exercício")
        print("0️⃣  Sair")

        opcao = input("Digite o número da opção desejada: ")

        if opcao == "1":
            nome = input("Nome: ")
            objetivo = input("Objetivo (ex: emagrecimento, hipertrofia): ")
            nivel = input("Nível (iniciante, intermediário, avançado): ")
            frequencia = input("Quantos dias por semana pretende treinar? ")
            equipamento = input("Equipamentos disponíveis (ex: academia completa, halteres, nenhum): ")
            sexo = input("Sexo: ")
            altura = input("Altura (m): ")
            peso = input("Peso (kg): ")

            perfil = {
                "nome": nome,
                "objetivo": objetivo,
                "nivel": nivel,
                "frequencia": frequencia,
                "equipamento": equipamento,
                "sexo": sexo,
                "altura": altura,
                "peso": peso
            }

            print("\n📋 Gerando plano de treino personalizado...\n")
            plano = agente_treino(perfil)
            plano_salvo = plano  # Armazena o plano gerado para uso no agente 3
            display(Markdown(f"### 🏋️ Plano de Treino para {nome}\n{plano}"))

        elif opcao == "2":
            exercicio = input("Digite o nome do exercício: ")
            resposta = agente_execucao_exercicio(exercicio)
            print("\n📽️ Execução do exercício:\n")
            display(to_markdown(resposta))

        elif opcao == "3":
            exercicio = input("Digite o nome do exercício que deseja substituir: ")
            nivel = input("Qual seu nível (iniciante, intermediário, avançado)? ")
            equipamento = input("Equipamentos disponíveis (ex: academia completa, halteres, nenhum): ")
            print("\n🔄 Buscando substituições para o exercício...\n")
            resposta = agente_substituicao_exercicio(exercicio, nivel, equipamento)
            display(to_markdown(resposta))

        elif opcao == "0":
            print("Encerrando o sistema. Até logo! 👋")
            break

        else:
            print("❌ Opção inválida. Tente novamente.")


In [ ]:
menu_principal()